In [1]:
# Подгрузим необходимые библиотеки 
import requests
from time import sleep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Парсинг 

Работа с API сайта <a target="_blank" href="hh.ru">hh.ru</a>. Документация доступна по <a target="_blank" href="https://github.com/hhru/api/blob/master/docs/vacancies.md#search">ссылке</a>.

**Задача:**
1. выберем 5 не смежныъ профессий.
2. затем получим доступные по данным профессиям вакансии в Москве и выделим их физические координаты.
3. Отберем вакансии в пределах МКАД, нанесем полученные координаты на карту местоположения компаний.

In [3]:
URL = 'https://api.hh.ru/vacancies'
jobs = ['Тестировщик', 'Инженер', 'Графический дизайнер', 'Преподаватель английского языка', 'Аналитик']
data = []
vacancy_counters = []
for vacancy in jobs:
    req = requests.get(URL,
                       {'text': vacancy, 'area': 1, 'page': 0, 'per_page': 100 })
    if req.status_code != 200:
        continue 
    df = pd.json_normalize(req.json()['items'])
    df['vacancy'] = vacancy
    data.append(df)
    vacancy_counters.append(req.json()['found'])
    for i in range(1, req.json()['pages']):
        req = requests.get(URL,
                           {'text': vacancy, 'area': 1, 'page': i, 'per_page': 100 })
        if req.status_code != 200:
            continue 
        df = pd.json_normalize(req.json()['items'])
        df['vacancy'] = vacancy
        data.append(df)
        
data = pd.concat(data, ignore_index = True)
data.head()

,id,premium,name,has_test,response_letter_required,salary,address,response_url,sort_point_distance,published_at,...,address.metro.station_id,address.metro.line_id,address.metro.lat,address.metro.lng,address.metro_stations,address.id,insider_interview.id,insider_interview.url,address.metro,vacancy
0,107313304,False,QA Engineer,False,False,NaN,NaN,None,None,2024-09-16T10:55:52+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тестировщик
1,106353373,False,Frontend разработчик (React),False,False,NaN,NaN,None,None,2024-08-26T19:19:49+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тестировщик
2,107284212,False,"QA Тестировщик (web, desktop и mobile)",False,False,NaN,NaN,None,None,2024-09-15T13:49:00+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тестировщик
3,107283779,False,Инженер по тестированию ПО,True,False,NaN,NaN,None,None,2024-09-15T12:38:16+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тестировщик
4,107328483,False,QA Engineer/тестировщик,False,False,NaN,NaN,None,None,2024-09-16T13:19:41+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тестировщик


Узнаем сколько всего найдено вакансий:

In [4]:
pd.DataFrame({'job':jobs, 'count': vacancy_counters})

,job,count
0,Тестировщик,2550
1,Инженер,23871
2,Графический дизайнер,1531
3,Преподаватель английского языка,438
4,Аналитик,23520


Найдем количество пропусков в данных о координатах:

In [5]:
data['address.lng'].isnull().sum()

3828

In [6]:
data['address.lat'].isnull().sum()

3828

Очистим данные от пропусков и выберем интересующие нас признаки:

In [7]:
data = data[['employer.name', 'address.lat','address.lng']]
data = data.dropna()
data.head()

,employer.name,address.lat,address.lng
5,АйТи БАСТИОН,55.707791,37.595806
9,R-Style Softlab (Эр-Стайл Софтлаб),55.623315,37.511041
20,Инфорус,55.734638,37.567698
22,Content AI,55.856563,37.599507
24,БИФИТ,55.791030,37.808761


Найдем количество вакансий, у которых заданы координаты:

In [8]:
data.shape

(4142, 3)

</i> у 4142 компаний заданы координаты, те компании , у которых координаты были пустими, очищены на предыдущем шаге. <i>

Отберем вакансии, которые расположены в пределах МКАД:

Напишем функцию, которая рассчитывает расстояние между объектами, знаяя их координаты, для каждого объекта найдем его расстояние до Кремля.

In [10]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Радиус Земли в километрах
    R = 6371.0

    # Конвертируем координаты из градусов в радианы
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Разности координат
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Формула Хаверсина
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Возврат расстояния в километрах
    distance = R * c
    return distance

fixed_lat, fixed_lng = 55.752004, 37.617734 # Координаты кремля


data['distance'] = data.apply(lambda x: haversine(x['address.lat'], x['address.lng'],
                               fixed_lat, fixed_lng),axis = 1)


In [11]:
# Будем считать, что расстояние от кремля до МКАДа равно 15 км.
data = data[data.distance <= 15]


In [12]:
data = data.rename(columns = {'employer.name': 'company',
                     'address.lat': 'latitude', 'address.lng' :'longitude' })

Построим график в координатах "широта-долгота" для отображения вакансий внутри МКАД:

In [21]:
import plotly.express as px



fig = px.scatter_mapbox(
    data,
    lat='latitude',
    lon='longitude',
    hover_name='company',
    opacity=0.6,  
    title='Вакансии внутри МКАД',
    mapbox_style='carto-positron', 
    zoom=10,  
    height=600
)


# fig.write_html("grafik.html")
fig.show()

**Выводы о проделанной работе:**

При парсинге данных довольно часто возникают проблемы. Было найдено намного больше вакансий, чем по итогу удалось прочитать. Причем в прочитанных данных было довольно много пропусков. По пулученному в ходе работы графику можно сделать вывод о том, что большинство компаний сконцентрированно в центре.